In [22]:
import sys
sys.path.append('/share/gpu0/jjwhit/rcGAN/')
import numpy as np
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from models.lightning.mmGAN import mmGAN
from models.archs.mri.generator import UNetModel
from models.archs.mri.discriminator import DiscriminatorModel
from data.lightning.MassMappingDataModule import MMDataTransform

import yaml
import json
import types
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:


examples_path = '/share/gpu0/jjwhit/mass_map_dataset/kappa20_debug/kappa_val/kappa_run_00971.npy' 

args = {}

transform = MMDataTransform(args)

data = np.load(examples_path, allow_pickle=True).astype(np.complex128)
# Tranform data and generate observations
data_transformed = transform(data)

In [32]:

y, x, mean, std = data_transformed

y = y.to(device)

In [34]:
y.shape

torch.Size([2, 1024, 1024])

In [35]:
y = y[None,:,:,:]
y.shape

torch.Size([1, 2, 1024, 1024])

In [36]:
def get_noise(num_vectors):
    z = torch.randn(num_vectors, 2, 1024, 1024, device=device)
    return z

num_vectors = y.size(0)

noise = get_noise(num_vectors)

In [37]:
gen_input = torch.cat([y, noise], dim=1)

In [44]:
gen_input.dtype

torch.float32

In [47]:
gen_input = gen_input.type(torch.FloatTensor)

In [39]:
from models.archs.mri.generator import UNetModel


In [48]:
in_chans = 2 + 2
out_chans = 2

generator = UNetModel(
    in_chans=in_chans,
    out_chans=out_chans,
)

In [49]:

samples = generator(gen_input)



In [50]:
samples.shape

torch.Size([1, 2, 1024, 1024])

In [ ]:
reformatted_tensor = reformat(samples)

## Readd_measures Function

In [ ]:
def reformat(self, samples):
    reformatted_tensor = torch.zeros(size=(samples.size(0), 1, self.resolution, self.resolution, 2),
                                     device=self.device)
    #Takes values from samples and assigns to reformatted tensor
    #assumption: 0:8 for real, 8:16 for complex, multiple elements bc multiple MRI slices?
    reformatted_tensor[:, :, :, :, 0] = samples[:, 0, :, :]
    reformatted_tensor[:, :, :, :, 1] = samples[:, 1, :, :]

    return reformatted_tensor

In [ ]:
def readd_measures(self, samples, measures):
    reformatted_tensor = self.reformat(samples)
    measures = fft2c_new(self.reformat(measures))
    reconstructed_kspace = fft2c_new(reformatted_tensor)

    # reconstructed_kspace = mask * measures + (1 - mask) * reconstructed_kspace

    image = ifft2c_new(reconstructed_kspace)

    output_im = torch.zeros(size=samples.shape, device=self.device)
    output_im[:, 0, :, :] = image[:, :, :, :, 0]
    output_im[:, 1, :, :] = image[:, :, :, :, 1]

    return output_im

In [55]:
reformatted_tensor = torch.zeros(size=(5, 1024, 1024, 2), device=device)

ex_samples = torch.zeros(size=(5, 2, 1024, 1024), device=device)




In [56]:

ex_samples[:, 0, :, :].shape


torch.Size([5, 1024, 1024])

In [57]:

reformatted_tensor[:, :, :, 0].shape


torch.Size([5, 1024, 1024])

In [58]:
ex_samples.shape

torch.Size([5, 2, 1024, 1024])

In [59]:
torch.mean(ex_samples, dim=1).shape

torch.Size([5, 1024, 1024])